In [8]:
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import logging
import ibis as ib
import geopandas as gpd
import nird.road_revised as func
from nird.utils import load_config
import json

ib.options.interactive = True

base_path = Path(load_config()['paths']['soge_clusters'])
warnings.simplefilter('ignore')
tqdm.pandas()

In [10]:
def main(
    depth_key,
    event_key,
    number_of_cpu,
):
    # Load network parameters
    with open(base_path / "parameters" / "flow_breakpoint_dict.json", "r") as f:
        flow_breakpoint_dict = json.load(f)
        
    for day in range(0, 1): # 0-12
        logging.info("Loading inputs...")
        road_links = ib.read_parquet(base_path.parent / "results" / "disruption_analysis" / "revision" / "ibis_results" / f"road_links_depth{depth_key}_event{event_key}_day{day}.gpq")
        road_links = road_links.cast({'geometry' : 'GEOMETRY'}).execute().set_geometry('geometry').set_crs('epsg:27700')
        if day == 0: # event_day
            road_links['acc_speed'] = road_links[['acc_speed', 'max_speed']].min(axis=1) 
        disrupted_od = pd.read_parquet(base_path.parent / "results" / "disruption_analysis" / "revision" / "ibis_results" / f"disrupted_od_depth{depth_key}_event{event_key}_day{day}.pq")
        disrupted_od.flow = disrupted_od.flow.fillna(0)
        disrupted_od["Car21"] = (disrupted_od.flow - disrupted_od.min_link_capacity).clip(lower=0) # disrupted flows
        disrupted_od.Car21 = disrupted_od.Car21.round(0).astype(int)
        
        total_cost = (
            disrupted_od.Car21 * 
            (
                disrupted_od.operating_cost_per_flow +
                disrupted_od.time_cost_per_flow +
                disrupted_od.toll_cost_per_flow
            )
        ).sum()
        
        logging.info("Creating network...")
        valid_road_links = road_links[(road_links.acc_capacity > 0) & (road_links.acc_speed >0)].reset_index(drop=True)
        network, valid_road_links = func.create_igraph_network(valid_road_links)

        logging.info("Running flow simulation...")
        logging.info(f"The total disrupted flow is: {disrupted_od.Car21.sum()}")
        valid_road_links, isolation, _, (_,_,_,total_cost2) = func.network_flow_model(
            valid_road_links,
            network,
            disrupted_od[["origin_node", "destination_node", "Car21"]],
            flow_breakpoint_dict,
            num_of_cpu = number_of_cpu,
        )
        logging.info(f"The total pre-event cost: £ million {total_cost/1e6}")
        logging.info(f"The total after-event cost: £ million {total_cost2/1e6}")
        rerouting_cost = total_cost2 - total_cost
        logging.info(f"The rerouting cost for depth{depth_key}-event{event_key}-scenario{day}: £ million{rerouting_cost/1e6}")
        logging.info("Saving results to disk...")
        out_path = (
            base_path.parent
            / "results"
            / "rerouting_analysis"
            / "revision"
            / str(depth_key)  # e.g.,30
            / str(event_key)  # e.g.,17
        )
        out_path.mkdir(parents=True, exist_ok=True)
        # isolations
        isolation_df = pd.DataFrame(isolation, columns = ["origin_node", "destination_node", "Car21"])
        isolation_df.to_csv(out_path / f"trip_isolations_{day}.csv", index=False)
        # edge flows
        road_links = road_links.set_index("e_id")
        road_links.update(valid_road_links.set_index("e_id")["acc_flow"])
        road_links.reset_index(drop = True, inplace = True)
        road_links["change_flow"] = road_links.acc_flow - road_links.current_flow
        road_links.to_parquet(out_path / f"edge_flows_{day}.gpq", engine="pyarrow")

        del valid_road_links
        del road_links
        del disrupted_od
    
    # rerouting costs over all scenarios (days)
    cDict[day] = rerouting_cost
    cost_df = pd.DataFrame.from_dict(cDict, orient="index", columns=["rerouting_cost"]).reset_index()
    cost_df.rename(columns = {"index": "scenario"}, inplace =True)
    cost_df.to_csv(out_path / "rerouting_cost.csv", index=False)

if __name__ == "__main__":
    logging.basicConfig(
        format="%(asctime)s %(process)d %(filename)s %(message)s", level=logging.INFO
    )
    main(depth_key =15, event_key=5, number_of_cpu=20)

2025-09-18 11:08:32,018 1969364 1100760503.py Loading inputs...
2025-09-18 11:08:44,044 1969364 1100760503.py Creating network...
2025-09-18 11:10:04,592 1969364 1100760503.py Running flow simulation...
2025-09-18 11:10:04,593 1969364 1100760503.py The total disrupted flow is: 12489
2025-09-18 11:10:04,610 1969364 road_revised.py The initial supply is 12489
2025-09-18 11:10:04,799 1969364 road_revised.py The initial number of edges in the network: 500585
2025-09-18 11:10:04,841 1969364 road_revised.py The initial number of origins: 45663
2025-09-18 11:10:04,877 1969364 road_revised.py The initial number of destinations: 33129
2025-09-18 11:10:04,878 1969364 road_revised.py No.1 iteration starts:
2025-09-18 11:10:05,472 1969364 road_revised.py Initial isolated flows: 2081
Creating argument list: 100%|██████████| 44099/44099 [07:13<00:00, 101.61it/s]
2025-09-18 11:17:25,414 1969364 road_revised.py Completed 0 of 44099, 0.00%
2025-09-18 11:24:30,877 1969364 road_revised.py Completed 10000

No low saturation edges found.


2025-09-18 11:50:48,971 1969364 road_revised.py Updating edge speeds: 
100%|██████████| 500585/500585 [00:06<00:00, 80295.05it/s]
2025-09-18 11:50:55,623 1969364 road_revised.py The total remain flow (after adjustment) is: 0.0.
2025-09-18 11:50:55,641 1969364 road_revised.py Stop: 100.0% of flows (exc. isolations) have been allocated and there is no edge overflow!
2025-09-18 11:50:55,813 1969364 road_revised.py The flow simulation is completed!
2025-09-18 11:50:55,814 1969364 road_revised.py total travel cost is (£): 441656.74285544443
2025-09-18 11:50:55,814 1969364 road_revised.py total time-equiv cost is (£): 438144.68142100604
2025-09-18 11:50:55,815 1969364 road_revised.py total operating cost is (£): 3512.0614344383666
2025-09-18 11:50:55,816 1969364 road_revised.py total toll cost is (£): 0.0
2025-09-18 11:50:58,022 1969364 1100760503.py The total pre-event cost: £ million 0.08030125536686196
2025-09-18 11:50:58,025 1969364 1100760503.py The total after-event cost: £ million 0.4

NameError: name 'cDict' is not defined